In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
df = df.rename(columns={'labels':'label','msgs':'message','diffs':'diff'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_3808/1424687940.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,repo,commit,label,message,diff,feature
user,,,,,,
ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,2,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,0,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,0,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ..."
ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,1,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,..."
ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,1,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...
jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,2,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,2,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,..."
hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,2,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17..."


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/1793/eval.csv',index=False)

In [5]:
train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=15, random_state=42))

/tmp/ipykernel_3808/464921656.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=15, random_state=42))


In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train

,repo,commit,label,message,diff,feature
user,,,,,,
elastic,elasticsearch,95ba62f83dfa05990d2165484330cdd0792064d8,0,Translog: Implement a file system based transl...,diff --git a/modules/elasticsearch/src/main/ja...,"[1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
apache,hbase,a8806266b898ee71376ce9722de407ece9ed359a,0,HBASE-1200 Add bloomfilters--git-svn-id: https...,diff --git a/.gitignore b/.gitignore\nindex 42...,"[18, 374, 58, 102, 120, 32, 24, 8, 138, 1, 31,..."
aeshell,aesh,60dfbfe5d954228d404eebc435346172cfbadec1,0,LineParser now supports curly brackets. It hav...,diff --git a/src/main/java/org/aesh/command/im...,"[7, 121, 54, 22, 1, 0, 0, 1, 23, 0, 62, 0, 0, ..."
elastic,elasticsearch,30f9f278c3430f9e936f566ee8b3394f86f2b01e,0,Added UNICODE_CHARACTER_CLASS support to Regex...,diff --git a/src/main/java/org/elasticsearch/c...,"[1, 8, 0, 8, 5, 0, 0, 1, 36, 0, 0, 18, 0, 0, 0..."
GNOME,valadoc,8de9caa985fa3e79348c1c88b9313a4bce4d92e6,0,libvaladoc: Add support for SourceCode attribu...,diff --git a/src/libvaladoc/content/sourcecode...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
GNOME,vala,42393d7aacf98e0e1657aadebdc8d4961db7a447,0,codegen: Inherit array_length and array_null_t...,diff --git a/codegen/valaccodeattribute.vala b...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
JetBrains,kotlin,31f4ff749c34c8ff573a1d6e1688c39ea1b4463c,0,Type annotations supported in Java elements--R...,diff --git a/compiler/frontend.java/src/org/je...,"[10, 84, 15, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
JetBrains,intellij-community,7bb429c9cde94494cde83b99715149c680c1dae9,0,external system: add clickable link to log- fo...,diff --git a/platform/external-system-api/reso...,"[1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
JetBrains,intellij-community,fbe88fe1a56907fae90cdf3ebf8c9fb1e12c7d09,0,form scope provider & dom lookups implemented--,diff --git a/codeInsight/impl/com/intellij/cod...,"[5, 38, 10, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [8]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)


base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 1.1387: 100%|██████████| 6/6 [00:03<00:00,  1.73batch/s]


=============================train========================


Epoch 2/10 Loss: 0.8558: 100%|██████████| 6/6 [00:02<00:00,  2.11batch/s]


=============================train========================


Epoch 3/10 Loss: 0.6182: 100%|██████████| 6/6 [00:02<00:00,  2.13batch/s]


...

=============================train========================


Epoch 9/10 Loss: 0.0528: 100%|██████████| 6/6 [00:02<00:00,  2.10batch/s]


=============================train========================


Epoch 10/10 Loss: 0.0281: 100%|██████████| 6/6 [00:02<00:00,  2.10batch/s]


=============================train========================
Validation Accuracy: 0.5693
Precision: 0.5762
Recall: 0.5693
F1-Score: 0.5556
              precision    recall  f1-score   support

           0     0.6182    0.3820    0.4722        89
           1     0.5615    0.8111    0.6636        90
           2     0.5488    0.5114    0.5294        88

    accuracy                         0.5693       267
   macro avg     0.5762    0.5682    0.5551       267
weighted avg     0.5762    0.5693    0.5556       267



In [9]:
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.5448
Precision: 0.5573
Recall: 0.5448
F1-Score: 0.5364
              precision    recall  f1-score   support

           0     0.5556    0.3797    0.4511        79
           1     0.5000    0.7416    0.5973        89
           2     0.6098    0.5000    0.5495       100

    accuracy                         0.5448       268
   macro avg     0.5551    0.5404    0.5326       268
weighted avg     0.5573    0.5448    0.5364       268



In [11]:
cm = confusion_matrix(test_labels, test_predictions)
cm

NameError: name 'confusion_matrix' is not defined

In [ ]:
# test_data

In [ ]:
、